In [1]:
%load_ext lab_black
%cd ../../..

/mnt/h/hev/log-analytics


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from collections import defaultdict
import sys

In [3]:
distdeck = torch.load(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-distdeck.pth"
)

In [4]:
deck2 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-deck2.pth")

In [5]:
df = pd.read_csv("data/ori/test.csv")

In [6]:
distdeck["dist"].shape, distdeck["level"].shape

(torch.Size([1095951, 4]), torch.Size([1095951, 4]))

In [7]:
distdeck.keys(), deck2.keys()

(dict_keys(['dist', 'level']), dict_keys(['feat', 'otext', 'fclevel', 'ids']))

In [8]:
total_len = 1418916

In [9]:
def policy(dists, tlevels, fclevel):
    if fclevel in [6, 4, 2]:
        return fclevel.item()
    if (tlevels == 5).all():
        return 5 if dists[0] < 1.5 else 7
    if (tlevels == 3).all():
        return 3 if dists[0] < 1.5 else 7
    if dists[0] < 0.7:
        # return tlevels[0].item()
        return fclevel.item()
    return 7

In [10]:
out_dists = [None for _ in range(total_len)]
out_levels = [None for _ in range(total_len)]
out_fclevels = [None for _ in range(total_len)]
N = distdeck["dist"].size(0)
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dists = distdeck["dist"][i]
        levels = distdeck["level"][i]
        fclevel = deck2["fclevel"][i]
        out_level = policy(dists, levels, fclevel)
        ids = deck2["ids"][i]
        for j in ids:
            out_levels[j - 1000000] = out_level
            out_dists[j - 1000000] = dists
            out_fclevels[j - 1000000] = fclevel
        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [01:40<00:00, 10942.63it/s]


In [11]:
out_levels = np.array(out_levels)

In [12]:
# 각 레벨별 개수 0.5이하일 때 fclevel을 쓰기로
for i in range(8):
    cnt = (out_levels == i).sum()
    print(i, ":", cnt, f"{cnt / len(out_levels)*100:.2f}%")

0 : 1002253 70.64%
1 : 396259 27.93%
2 : 42 0.00%
3 : 12972 0.91%
4 : 34 0.00%
5 : 6342 0.45%
6 : 29 0.00%
7 : 985 0.07%


```
0 : 1003955 70.76%
1 : 395007 27.84%
2 : 42 0.00%
3 : 12950 0.91%
4 : 34 0.00%
5 : 6334 0.45%
6 : 31 0.00%
7 : 563 0.04%
```

In [13]:
(out_levels == 6).nonzero()

(array([  35574,   73308,   86431,   92457,  147553,  175492,  179014,
         210545,  240872,  340297,  355854,  374455,  451522,  484782,
         499683,  507877,  529914,  624698,  633202,  647782,  652696,
         714497,  848132,  881660, 1113383, 1125342, 1169189, 1293703,
        1394234]),)

In [14]:
xdf = pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver2.csv")

In [15]:
xlevels = xdf.level.to_numpy()

In [16]:
(xlevels == 6).nonzero()

(array([  35574,   64268,   73308,   86431,   92457,  147553,  175492,
         179014,  210545,  240872,  340297,  355854,  374455,  451522,
         484782,  499683,  507877,  529914,  563330,  624698,  633202,
         647782,  652696,  714497,  848132,  881660, 1113383, 1125342,
        1169189, 1293703, 1394234]),)

In [17]:
df.full_log[64268]

'Sep 18 13:17:49 localhost useradd[12075]: new group: name=suricata, GID=1001'

In [18]:
tdf = pd.read_csv("data/ori/train.csv")

In [19]:
otexts = tdf.full_log.to_numpy()

In [22]:
def find(text):
    out_indices = []
    for i, otext in enumerate(otexts):
        if text in otext:
            return i


def find_all(text):
    out_indices = []
    for i, otext in enumerate(otexts):
        if text in otext:
            out_indices.append(i)
    return out_indices

In [23]:
find_all("new group")

[221847]

In [25]:
tdf.level[221847], tdf.full_log[221847]

(6, 'Jan 29 11:28:59 localhost useradd[88679]: new group: name=test, GID=1001')

In [36]:
for i in (xlevels == 6).nonzero()[0]:
    print(f"{i:8} | {str(df.full_log[i])}")

   35574 | Jan 20 18:43:36 localhost sshd[2106]: Bad protocol version identification '\003' from 185.176.220.106 port 65322
   64268 | Sep 18 13:17:49 localhost useradd[12075]: new group: name=suricata, GID=1001
   73308 | Feb  2 16:55:27 localhost kernel: device enp2s0 entered promiscuous mode
   86431 | Mar  8 15:29:20 localhost kernel: device virbr0-nic entered promiscuous mode
   92457 | Feb  2 17:06:35 localhost kernel: device enp2s0 entered promiscuous mode
  147553 | Nov 18 14:13:50 localhost sshd[15801]: error: maximum authentication attempts exceeded for root from 192.168.0.195 port 33620 ssh2 [preauth]
  175492 | Jan 30 19:28:40 localhost sshd[29475]: Bad protocol version identification '\003' from 185.176.220.106 port 63400
  179014 | Jan 29 11:28:59 localhost useradd[88679]: new user: name=test, UID=1001, GID=1001, home=/home/test, shell=/bin/bash
  210545 | Jan 12 16:19:13 localhost kernel: device enp2s0 entered promiscuous mode
  240872 | Jan 22 15:19:23 localhost sshd[19

In [37]:
for i in (out_levels == 6).nonzero()[0]:
    print(f"{i:8} | {str(df.full_log[i])}")

   35574 | Jan 20 18:43:36 localhost sshd[2106]: Bad protocol version identification '\003' from 185.176.220.106 port 65322
   73308 | Feb  2 16:55:27 localhost kernel: device enp2s0 entered promiscuous mode
   86431 | Mar  8 15:29:20 localhost kernel: device virbr0-nic entered promiscuous mode
   92457 | Feb  2 17:06:35 localhost kernel: device enp2s0 entered promiscuous mode
  147553 | Nov 18 14:13:50 localhost sshd[15801]: error: maximum authentication attempts exceeded for root from 192.168.0.195 port 33620 ssh2 [preauth]
  175492 | Jan 30 19:28:40 localhost sshd[29475]: Bad protocol version identification '\003' from 185.176.220.106 port 63400
  179014 | Jan 29 11:28:59 localhost useradd[88679]: new user: name=test, UID=1001, GID=1001, home=/home/test, shell=/bin/bash
  210545 | Jan 12 16:19:13 localhost kernel: device enp2s0 entered promiscuous mode
  240872 | Jan 22 15:19:23 localhost sshd[19037]: error: maximum authentication attempts exceeded for root from 192.168.0.197 port 6

In [38]:
for i, (level, otext) in enumerate(zip(tdf.level, tdf.full_log)):
    if level == 6:
        print(f"{i:8} | {otext}")

   87783 | Jan 30 08:23:17 localhost sshd[18415]: Bad protocol version identification '\003' from 78.128.113.18 port 1073
  170386 | Feb  8 16:16:47 localhost kernel: device virbr0-nic entered promiscuous mode
  213304 | Mar  8 15:29:30 localhost kernel: device enp2s0 entered promiscuous mode
  221847 | Jan 29 11:28:59 localhost useradd[88679]: new group: name=test, GID=1001
  223644 | Jan 22 15:18:49 localhost sshd[19015]: error: maximum authentication attempts exceeded for root from 192.168.0.197 port 61153 ssh2 [preauth]
  361823 | Feb  2 17:22:58 localhost kernel: device virbr0-nic entered promiscuous mode
  406796 | Feb  4 09:59:33 localhost kernel: device virbr0-nic entered promiscuous mode
  429489 | Feb  4 09:59:43 localhost kernel: device enp2s0 entered promiscuous mode


---

In [39]:
out_ids = list(range(1000000, 1000000 + len(out_levels)))

In [40]:
out_df = {"id": out_ids, "level": out_levels}

In [41]:
out_df = pd.DataFrame(out_df)

In [42]:
out_df

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [47]:
# 0.9198
out_df.to_csv(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-out_ver1.csv", index=False
)

In [43]:
# ver1보다 threshold를 좀 더 만만하게 준 버전 - 0.9207
out_df.to_csv(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-out_ver2.csv", index=False
)

In [22]:
# level 3, 5를 꼭 모두 3, 5이지 않아도 되게 설정 - 0.9189
out_df.to_csv(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os40_2-out_ver3.csv", index=False
)